# Linelist generator

This is a simple notebook to generate a linelist for use by `cloudy`.

In [57]:
import numpy as np
from synthesizer.photoionisation import cloudy23

In [58]:

wavelength_type = 'standard'
wavelength_type = 'vacuum'

# the reference line to use
if wavelength_type == 'standard':
    reference_line = 'H 1 4861.32A'
if wavelength_type == 'vacuum':
    reference_line = 'H 1 4862.69A'

# log constrast relative to Hbeta to consider
contrast_stars = 1.0
contrast_agn = 1.0

# minimum wavelength of lines to consider
min_wavelength = 1000

# maximum wavelength of lines to consider
max_wavelength = 25000

### Stellar models

In [59]:

linelist = []

model = r'metallicity_'+wavelength_type
for i in range(4):
    i += 1
    line_ids, blends, wavelengths, intrinsic, emergent = cloudy23.read_lines(rf'{model}/{i}')
    Hbeta = emergent[line_ids==reference_line][0]
    s = (emergent > (Hbeta - contrast_stars)) & (wavelengths<max_wavelength) & (wavelengths>min_wavelength) & (blends == False)
    print(i, np.sum(s))
    linelist += list(line_ids[s])



1 34
2 30
3 15
4 9


### AGN models

In [60]:

# model = r'relagn_'+wavelength_type

# for i in range(3):
#     i += 1
#     line_ids, blends, wavelengths, intrinsic, emergent = cloudy23.read_lines(rf'{model}/{i}')
#     Hbeta = emergent[line_ids==reference_line][0]
#     s = (emergent > (Hbeta - contrast_agn)) & (wavelengths<max_wavelength) & (wavelengths>min_wavelength) & (blends == False)
#     print(i, np.sum(s))
#     linelist += list(line_ids[s])


In [61]:
linelist = list(set(linelist))
linelist.sort()

# print(linelist)
print(len(linelist))

linelist_ = []

for line in linelist:

    element, ion, wavelength = line.split(' ')

    if len(ion)==1:

        line_ = f'{element} {ion} {wavelength}'
        print(line_)
        linelist_.append(line_)


with open(f'linelist-{wavelength_type}.dat', 'w') as file:
    file.writelines('\n'.join(linelist_) + '\n')

39
Ar 3 7137.76A
C 3 1906.68A
C 3 1908.73A
C 4 1548.19A
C 4 1550.78A
Fe 2 1.25702m
Fe 2 1.64400m
Fe 4 2836.57A
Fe 4 3095.86A
H 1 1.09410m
H 1 1.28215m
H 1 1.87561m
H 1 1215.67A
H 1 3890.15A
H 1 3971.19A
H 1 4102.89A
H 1 4341.68A
H 1 4862.69A
H 1 6564.62A
He 1 1.08332m
He 1 3889.74A
He 1 5877.26A
Mg 2 2796.35A
Mg 2 2803.53A
N 2 6549.86A
N 2 6585.27A
Ne 3 3869.86A
Ne 3 3968.59A
O 2 3727.09A
O 2 3729.88A
O 3 1666.15A
O 3 4960.29A
O 3 5008.24A
S 2 6718.29A
S 2 6732.67A
S 3 9071.11A
S 3 9533.23A
Si 3 1882.71A
Si 3 1892.03A
